# Line plot, performance to target

# Document information

<table align="left">
    <tr>
        <th class="text-align:left">Title</th>
        <td class="text-align:left">Line plot, performance to target</td>
    </tr>
    <tr>
        <th class="text-align:left">Last modified</th>
        <td class="text-align:left">2091-01-24</td>
    </tr>
    <tr>
        <th class="text-align:left">Author</th>
        <td class="text-align:left">Gilles Pilon <gillespilon13@gmail.com></td>
    </tr>
    <tr>
        <th class="text-align:left">Status</th>
        <td class="text-align:left">Active</td>
    </tr>
    <tr>
        <th class="text-align:left">Type</th>
        <td class="text-align:left">Jupyter notebook</td>
    </tr>
    <tr>
        <th class="text-align:left">Created</th>
        <td class="text-align:left">2019-01-24</td>
    </tr>
    <tr>
        <th class="text-align:left">File name</th>
        <td class="text-align:left">line_plot_actual_vs_target.ipynb</td>
    </tr>
    <tr>
        <th class="text-align:left">Other files required</th>
        <td class="text-align:left">actual_vs_target.csv</td>
    </tr>
</table>

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('actual_vs_target.csv')

In [3]:
df.head()

,Date,Target,Actual
0,2019-01-31,10000,10000.0
1,2019-02-28,11500,NaN
2,2019-03-31,13000,NaN
3,2019-04-30,14500,NaN
4,2019-05-31,16000,NaN


This notebook plots two lines:

- Target value vs month of the year
- Actual value vs month of the year